In [ ]:
%pip install spacy_transformers

In [ ]:
%pip install -U spacy

In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [6]:
spacy.__version__

'3.6.1'

In [10]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!nvcc --version

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Load the annotated data from a JSON file
import json
cv_data = json.load(open("./CV-Parsing-Spacy3/data/training/train_data.json"))

In [3]:
len(cv_data)

200

In [ ]:
cv_data[1]

In [4]:
!python -m spacy init fill-config CV-Parsing-Spacy3/data/training/base_config.cfg CV-Parsing-Spacy3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
CV-Parsing-Spacy3\data\training\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True   # if entities are overlapping
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
        doc.ents = ents
        db.add(doc)
    except:
        pass

  return db


In [ ]:
%pip install scikit-learn

In [6]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.2)

In [7]:
# Display the number of items in the training and testing sets
len(train), len(test)

(160, 40)

In [8]:
# Open a file to log errors during annotation processing
file = open('./CV-Parsing-Spacy3/data/model_data/error_file.txt','w', encoding='utf-8')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('./CV-Parsing-Spacy3/data/model_data/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('./CV-Parsing-Spacy3/data/model_data/test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 40/40 [00:00<00:00, 85.29it/s] 


In [9]:
len(db.tokens)

40

In [ ]:
%pip uninstall cupy-cuda12x

In [5]:
!pip freeze

altair==5.1.1
altgraph==0.17.3
annotated-types==0.5.0
anyio==3.7.1
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
async-lru==2.0.3
attrs==23.1.0
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.12.2
bleach==6.0.0
blinker==1.6.2
blis==0.7.11
cachelib==0.10.2
cachetools==5.3.1
catalogue==2.0.9
catboost==1.2.2
certifi==2023.7.22
cffi==1.15.1
charset-normalizer==3.2.0
click==8.1.7
colorama==0.4.6
comm==0.1.3
confection==0.1.3
contourpy==1.1.1
cycler==0.11.0
cymem==2.0.8
debugpy==1.6.7
decorator==5.1.1
defusedxml==0.7.1
executing==1.2.0
fastjsonschema==2.18.0
fastrlock==0.8.2
filelock==3.12.4
Flask==2.3.3
Flask-Cors==4.0.0
Flask-Session==0.5.0
fonttools==4.42.1
fqdn==1.5.1
fsspec==2023.9.2
future==0.18.3
gitdb==4.0.10
GitPython==3.1.36
graphviz==0.20.1
huggingface-hub==0.17.2
idna==3.4
importlib-metadata==6.8.0
ipykernel==6.25.0
ipython==8.14.0
ipython-genutils==0.2.0
ipywidgets==8.0.7
isoduration==20.11.0
itsdangerous==2.1.2
jedi==0.18.2
Jinja2==3.1.2
joblib=

In [7]:
%pip install cupy

  Using cached cupy-12.2.0.tar.gz (2.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [67 lines of output]
      Clearing directory: C:\Users\domin\AppData\Local\Temp\pip-install-wk4pnap4\cupy_c32cd5dd206043859ce07d759a83f313\cupy\.data
      Looking for NVTX: C:\Program Files\NVIDIA Corporation\Nsight Systems *\target-windows-x64\nvtx
      Using NVTX at: C:\Program Files\NVIDIA Corporation\Nsight Systems 2023.2.3\target-windows-x64\nvtx
      
      -------- Configuring Module: cuda --------
      Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      **************************************************
      *** WARNING: Cannot check compute capability
      Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      *******************************

In [8]:
!python -m spacy train CV-Parsing-Spacy3/data/training/config.cfg --output CV-Parsing-Spacy3/data/model_output --paths.train CV-Parsing-Spacy3/data/model_data/train_data.spacy --paths.dev CV-Parsing-Spacy3/data/model_data/test_data.spacy

^C


In [27]:
nlp = spacy.load(r"E:\Projects\AI Based CV Analysis Using ML\model_output\model-best")

In [29]:
doc = nlp("10 years of Experience in Designing, Development, Administration, Analysis, Management inthe Business Intelligence Data warehousing, Client Server Technologies, Web-based Applications, cloud solutions and Databases. Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific todata warehousing and business intelligence environment. Database: Experience in database designing, scalability, back-up and recovery, writing andoptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL). Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factoryto schedule U-SQL jobs. Designed and developed end to end big data solution for data insights.")
for ent in doc.ents:
    print(ent.text, "---", ent.label_)

+ years --- Years of Experience
Microsoft --- Companies worked at
